In [1]:
import numpy as np
import torch
from transformers import pipeline

# Look at code in DART

Some top models and their sizes.

**open-source**
- GPT-Neo 125 mil
- GPT-Neo 1.3 bil (same as GPT-3 Babbage)
- GPT-Neo 2.7 bil
- GPT-J 6 bil
- GPT-NeoX 20 bil
- Bloom: ranges from 350m to 176 bil

**closed-source**
- GPT-3: 175 bil at biggest

In [2]:
checkpoint = "EleutherAI/gpt-neo-125M"

# note: doesn't run on gpu by default
generator = pipeline('text-generation', model=checkpoint)
for text in ['test1']:
    generator(text, do_sample=True, min_length=20)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/home/chansingh/.local/lib/python3.8/site-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [16]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM


In [6]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


In [35]:
raw_inputs = [
    "It was good",
]
inputs = tokenizer(raw_inputs, return_tensors="pt")
inputs['input_ids'] = inputs['input_ids'].float()
inputs['input_ids'].requires_grad = True
print(inputs)

{'input_ids': tensor([[1026.,  373.,  922.]], requires_grad=True), 'attention_mask': tensor([[1, 1, 1]])}


In [36]:
inputs['input_ids'].dtype

torch.float32

In [37]:
model = AutoModelForCausalLM.from_pretrained(checkpoint)

outputs = model(**inputs)

loss = outputs['logits'].sum()
outputs['logits'].retain_grad()
loss.backward(retain_graph=True)
outputs['logits'].grad


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

In [38]:
vars(model)

{'training': False,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_post_hooks': OrderedDict(),
 '_modules': OrderedDict([('transformer',
               GPTNeoModel(
                 (wte): Embedding(50257, 768)
                 (wpe): Embedding(2048, 768)
                 (drop): Dropout(p=0.0, inplace=False)
                 (h): ModuleList(
                   (0): GPTNeoBlock(
                     (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
                     (attn): GPTNeoAttention(
                       (attention): GPTNeoSelfAttention(
                         (attn_dropout): Dropout(p=0.0, inplace=False)
                         (resid_dropout): Dropout(p=0.0, inplace=Fal

In [41]:
for k in model._modules:
    print(k)

transformer
lm_head


In [44]:
model._modules['lm_head']

Linear(in_features=768, out_features=50257, bias=False)

In [54]:

for i, mod in enumerate(model._modules['transformer'].modules()):
    if i > 0: # first mod contains whole net
        print(type(mod))

<class 'torch.nn.modules.sparse.Embedding'>
<class 'torch.nn.modules.sparse.Embedding'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'torch.nn.modules.container.ModuleList'>
<class 'transformers.models.gpt_neo.modeling_gpt_neo.GPTNeoBlock'>
<class 'torch.nn.modules.normalization.LayerNorm'>
<class 'transformers.models.gpt_neo.modeling_gpt_neo.GPTNeoAttention'>
<class 'transformers.models.gpt_neo.modeling_gpt_neo.GPTNeoSelfAttention'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.normalization.LayerNorm'>
<class 'transformers.models.gpt_neo.modeling_gpt_neo.GPTNeoMLP'>
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.linear.Linear'>
<class 'transformers.activations.NewGELUActivation'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'trans